In [1]:
# import CalculusWithJulia
using Plots
using Polynomials
# using RealPolynomialRoots
using DataFrames
using Random
using Printf
using PrettyTables
import SymPy
using Latexify
using LaTeXStrings

function rep!(expression::Expr, rules::Dict)
    for (i, a) in enumerate(expression.args)
        if a isa Expr
            rep!(a, rules)
        else
            for (old, new) in rules
                if a == old
                    expression.args[i] = new
                end
            end
        end
    end
    expression
end

verbose = true
print()

## Комп'ютерний практикум №1. Розв'язання нелінійних рівнянь.
### Костюковець Остап ФБ-06
### варіант 5

In [2]:
t = [-4, -2, 1, -3, 1]
t2 = [60,-43,-21,3,1]
my_coefs = [12, 0, 0, -1, -3, 4]

f = Polynomial(my_coefs)
P = Meta.parse("$f")
verbose ? display(@latexify f(x) = $P) : nothing


L"$f\left( x \right) = 12 - x^{3} - 3 \cdot x^{4} + 4 \cdot x^{5}$"

### Теорема Гюа

In [3]:
l = length(coeffs(f))
for i in 1:l
    if i + 2 > l
        break
    end
    k = coeffs(f)[i:i+2]
    if k[2]^2 < k[1] * k[3]
        verbose ? display(@latexify $(k[2])^2 < $(k[1]) * $(k[3])) : nothing
        verbose ? display(L"\text{ Існує хоча б одна пара комплексно спряжених коренів}") : nothing
        break
    end
    verbose ? display(@latexify $(k[2])^2 ≮ $(k[1]) * $(k[3])) : nothing
end



L"$\ 0^{2} \nless{12 \cdot 0}$"

L"$\ 0^{2} \nless{0 \cdot -1}$"

L"$\ \left( -1 \right)^{2} \nless{0 \cdot -3}$"

L"$\ \left( -3 \right)^{2} \nless{-1 \cdot 4}$"

### Кільце

In [4]:
A = maximum(abs.(coeffs(f)[1:end-1]))
verbose ? display(L"%$(@latexify A = max(abs(aᵢ))) , $i=0,...,n-1$ $=$ $%$A$") : nothing

B = maximum(abs.(coeffs(f)[2:end]))
verbose ? display(L"%$(@latexify B = max(abs(aᵢ))) , $i=1,...,n$ $=$ $%$B$") : nothing


a₀ = coeffs(f)[1]
r_plus_low_expr = :(abs(a₀) / (B + abs(a₀)))
R_plus_low = eval(r_plus_low_expr)

verbose ? display(@latexify $r_plus_low_expr = $(rep!(copy(r_plus_low_expr), Dict(:a₀=>coeffs(f)[1], :B=>B))) = $R_plus_low env=:eq fmt=FancyNumberFormatter(2)) : nothing

aₙ = coeffs(f)[end]
r_plus_high_expr = :((abs(aₙ) + A) / abs(aₙ))
R_plus_high = eval(r_plus_high_expr)

verbose ? display(@latexify $r_plus_high_expr = $(rep!(copy(r_plus_high_expr), Dict(:aₙ=>coeffs(f)[end], :A=>A))) =  $R_plus_high env=:eq fmt=FancyNumberFormatter(2)) : nothing

verbose ? display(@latexify $R_plus_low <= abs(x) <= $R_plus_high fmt=FancyNumberFormatter(2) env=:eq) : nothing


L"$A = \mathrm{max}\left( \left|a_i\right| \right)$ , $i=0,...,n-1$ $=$ $12$"

L"$B = \mathrm{max}\left( \left|a_i\right| \right)$ , $i=1,...,n$ $=$ $4$"

L"\begin{equation}
\frac{\left|a_0\right|}{B + \left|a_0\right|} = \frac{\left|12\right|}{4 + \left|12\right|} = 0.75
\end{equation}
"

L"\begin{equation}
\frac{\left|a_n\right| + A}{\left|a_n\right|} = \frac{\left|4\right| + 12}{\left|4\right|} = 4
\end{equation}
"

L"\begin{equation}
\left( 0.75 \leq \left|x\right| \leq 4 \right)
\end{equation}
"

### Верхня межа

In [5]:
# using Symbolics
# P1
P₁ = P
p1_coeffs = coeffs(f)
verbose ? display(@latexdefine P₁) : nothing

if p1_coeffs[end] < 0
    p1_coeffs = -1 .* p1_coeffs
    verbose ? display(@latexify P₁ = $(-P₁) = 0) : nothing
end

B = maximum(abs.([v for v in p1_coeffs if v < 0]))
verbose ? display(L"%$(@latexify B = max(abs(aᵢ))) , $aᵢ<0$ $=$ $%$B$") : nothing;

m = [i - 1 for (i, v) in enumerate(p1_coeffs) if v < 0][end]
verbose ? display(L"%$(@latexify m = max(i)) , $aᵢ<0$ $=$ $%$m$") : nothing

aₙ = p1_coeffs[end]
n = length(p1_coeffs) - 1
# r_plus_high_expr = :(1 + (B / aₙ)^(1 / (n - m)))
R₁_expr = :(1 + (B / aₙ)^(1 / (n - m)))
R₁ = eval(R₁_expr)
x_plus_high = R₁

verbose ? display(@latexify R₁ = $R₁_expr = $(rep!(copy(R₁_expr), Dict(:aₙ => p1_coeffs[end], :B => B, :n => n, :m => m))) = $R₁ env = :eq fmt = FancyNumberFormatter(2)) : nothing



#P2
SymPy.@syms x
p2_tmp = x^n * eval(rep!(copy(P), Dict(:x => :(1 / x))))
P₂ = SymPy.simplify(eval(p2_tmp))
verbose ? display(@latexify P₂ = x^n * P(1 / x) = $p2_tmp = $P₂) : nothing

x = variable()

p2_coeffs = coeffs(eval(Meta.parse(SymPy.repr(P₂))))

if p2_coeffs[end] < 0
    p2_coeffs = -1 .* p2_coeffs
    verbose ? display(@latexify P₂ = $(-P₂) = 0) : nothing
end

B = maximum(abs.([v for v in p2_coeffs if v < 0]))
verbose ? display(L"%$(@latexify B = max(abs(aᵢ))) , $aᵢ<0$ $=$ $%$B$") : nothing

m = [i - 1 for (i, v) in enumerate(p2_coeffs) if v < 0][end]
verbose ? display(L"%$(@latexify m = max(i)) , $aᵢ<0$ $=$ $%$m$") : nothing

aₙ = p2_coeffs[end]
n = length(p2_coeffs) - 1
R₂_expr = :(1 + (B / aₙ)^(1 / (n - m)))
R₂ = eval(R₂_expr)
x_plus_low = 1 / R₂

verbose ? display(@latexify R₂ = $R₂_expr = $(rep!(copy(R₂_expr), Dict(:aₙ => p2_coeffs[end], :B => B, :n => n, :m => m))) = $R₂ env = :eq fmt = FancyNumberFormatter(2)) : nothing


# P3
SymPy.@syms x
p3_tmp = eval(rep!(copy(P), Dict(:x => :(-x))))
P₃ = SymPy.simplify(p3_tmp)
verbose ? display(@latexify P₃ = P(-x) = $p3_tmp = $P₃) : nothing

x = variable()

p3_coeffs = coeffs(eval(Meta.parse(SymPy.repr(P₃))))

if p3_coeffs[end] < 0
    p3_coeffs = -1 .* p3_coeffs
    verbose ? display(@latexify P₃ = $(-P₃) = 0) : nothing
end

B = maximum(abs.([v for v in p3_coeffs if v < 0]))
verbose ? display(L"%$(@latexify B = max(abs(aᵢ))) , $aᵢ<0$ $=$ $%$B$") : nothing

m = [i - 1 for (i, v) in enumerate(p3_coeffs) if v < 0][end]
verbose ? display(L"%$(@latexify m = max(i)) , $aᵢ<0$ $=$ $%$m$") : nothing

aₙ = p3_coeffs[end]
n = length(p3_coeffs) - 1
R₃_expr = :(1 + (B / aₙ)^(1 / (n - m)))
R₃ = eval(R₃_expr)
x_minus_low = -R₃

verbose ? display(@latexify R₃ = $R₃_expr = $(rep!(copy(R₃_expr), Dict(:aₙ => p3_coeffs[end], :B => B, :n => n, :m => m))) = $R₃ env = :eq fmt = FancyNumberFormatter(2)) : nothing



#P4
SymPy.@syms x
p4_tmp = x^n * eval(rep!(copy(P), Dict(:x => :(-1 / x))))
P₄ = SymPy.simplify(eval(p4_tmp))
verbose ? display(@latexify P₄ = x^n * P(-1 / x) = $p4_tmp = $P₄) : nothing

x = variable()

p4_coeffs = coeffs(eval(Meta.parse(SymPy.repr(P₄))))

if p4_coeffs[end] < 0
    p4_coeffs = -1 .* p4_coeffs
    verbose ? display(@latexify P₄ = $(-P₄) = 0) : nothing
end

B = maximum(abs.([v for v in p4_coeffs if v < 0]))
verbose ? display(L"%$(@latexify B = max(abs(aᵢ))) , $aᵢ<0$ $=$ $%$B$") : nothing

m = [i - 1 for (i, v) in enumerate(p4_coeffs) if v < 0][end]
verbose ? display(L"%$(@latexify m = max(i)) , $aᵢ<0$ $=$ $%$m$") : nothing

aₙ = p4_coeffs[end]
n = length(p4_coeffs) - 1
R₄_expr = :(1 + (B / aₙ)^(1 / (n - m)))
R₄ = eval(R₄_expr)
x_minus_high = -1 / R₄

verbose ? display(@latexify R₄ = $R₄_expr = $(rep!(copy(R₄_expr), Dict(:aₙ => p4_coeffs[end], :B => B, :n => n, :m => m))) = $R₄ env = :eq fmt = FancyNumberFormatter(2)) : nothing


verbose ? display(L" %$(@latexify 1 / R₂ <= x⁺ <= R₁) ⇒ %$(@latexify $x_plus_low <= x⁺ <= $x_plus_high fmt = FancyNumberFormatter(2))") : nothing
verbose ? display(L" %$(@latexify -R₃ <= x⁻ <= -(1 / R₄)) ⇒ %$(@latexify $x_minus_low <= x⁻ <= $x_minus_high fmt = FancyNumberFormatter(2))") : nothing

print()

L"$P_1 = 12 - x^{3} - 3 \cdot x^{4} + 4 \cdot x^{5}$"

L"$B = \mathrm{max}\left( \left|a_i\right| \right)$ , $aᵢ<0$ $=$ $3$"

L"$m = \mathrm{max}\left( i \right)$ , $aᵢ<0$ $=$ $4$"

L"\begin{equation}
R_1 = 1 + \left( \frac{B}{a_n} \right)^{\frac{1}{n - m}} = 1 + \left( \frac{3}{4} \right)^{\frac{1}{5 - 4}} = 1.8
\end{equation}
"

L"$P_2 = x^{n} \cdot P\left( \frac{1}{x} \right) = x^{5} \cdot \left( 12 - \frac{1}{x^{3}} - \frac{3}{x^{4}} + \frac{4}{x^{5}} \right) = 12 \cdot x^{5} - x^{2} - 3 \cdot x + 4$"

...

L"$B = \mathrm{max}\left( \left|a_i\right| \right)$ , $aᵢ<0$ $=$ $4$"

L"$m = \mathrm{max}\left( i \right)$ , $aᵢ<0$ $=$ $1$"

L"\begin{equation}
R_4 = 1 + \left( \frac{B}{a_n} \right)^{\frac{1}{n - m}} = 1 + \left( \frac{4}{12} \right)^{\frac{1}{5 - 1}} = 1.8
\end{equation}
"

L" $\left( \frac{1}{R_2} \leq x^+ \leq R_1 \right)$ ⇒ $\left( 0.61 \leq x^+ \leq 1.8 \right)$"

L" $\left(  - R_3 \leq x^- \leq  - \frac{1}{R_4} \right)$ ⇒ $\left( -2.7 \leq x^- \leq -0.57 \right)$"

### Метод Штурма

In [6]:
f_tmp = copy(f)
d_array = []

f_idx = 0
while f_tmp != 0

    push!(d_array, f_tmp)

    f_expr = Meta.parse("$f_tmp")
    verbose ? display(L"$f_{%$f_idx} =$ %$(@latexify $f_expr)") : nothing
    f_idx += 1

    f_tmp = derivative(f_tmp)
end

interval_minus = [x_minus_low, x_minus_high]
interval_plus = [x_plus_low, x_plus_high]

data_frame_dict = Dict()
res = []

eg_counter = 0 #edge case counter

function bisect_sturm(interval)
    global eg_counter
    tmp = []
    for i in interval
        signs = [j(i) > 0 ? 1 : 0 for j in d_array]
        count = sum(k != signs[idx] for (idx, k) in enumerate(signs[2:end]))
        data_frame_dict[i] = push!(replace(signs, 1 => "+", 0 => "-"), count)
        push!(tmp, count)
    end

    if tmp[1] - tmp[2] <= 1
        push!(res, collect(zip(tmp, interval)))
        return
    end

    eg_counter += 1
    if eg_counter >= 5 # number of recursions allowed
        eg_counter = 0
        return
    end

    mid = 0
    if interval[1] < interval[2]
        mid = interval[2] + (interval[1] - interval[2]) / 2
    else
        mid = interval[1] + (interval[2] - interval[1]) / 2
    end

    bisect_sturm([interval[1], mid])
    bisect_sturm([mid, interval[2]])
end

bisect_sturm(interval_minus)
bisect_sturm(interval_plus)


target_intervals = [[itvl[1][2], itvl[2][2]] for itvl in res if itvl[1][1] != itvl[2][1]]


# table formatting 
_row_labels = []
for i in 0:(f_idx-1)
    push!(_row_labels, "f_{$i}")
end
push!(_row_labels, "")

_header = []
_data = []
for (key, value) in sort(data_frame_dict)
    push!(_header, @sprintf("%.3f", key))
    push!(_data, value)
end

_data = reduce(hcat, _data)


display(latexstring(replace(pretty_table(String,
        _data,
        header=_header,
        row_labels=_row_labels,
        hlines=:all,
        vlines=:all,
        alignment=:c,
        backend=Val(:latex),
    ),
    "tabular" => "array")))
# 

# result intervals
for (i, v) in enumerate(target_intervals)
    verbose ? display(@latexify $(v[1]) < x < $(v[2]) fmt = FancyNumberFormatter(2)) : nothing
end

L"$f_{0} =$ $12 - x^{3} - 3 \cdot x^{4} + 4 \cdot x^{5}$"

L"$f_{1} =$ $-3 \cdot x^{2} - 12 \cdot x^{3} + 20 \cdot x^{4}$"

L"$f_{2} =$ $-6 \cdot x - 36 \cdot x^{2} + 80 \cdot x^{3}$"

L"$f_{3} =$ $-6 - 72 \cdot x + 240 \cdot x^{2}$"

L"$f_{4} =$ $-72 + 480 \cdot x$"

L"$f_{5} =$ $480$"

L"$\begin{array}{|r|c|c|c|c|c|c|c|c|}
  \hline
   & \textbf{-2.732} & \textbf{-0.568} & \textbf{0.614} & \textbf{0.756} & \textbf{0.827} & \textbf{0.898} & \textbf{1.182} & \textbf{1.750} \\\hline
  f_{0} & - & + & + & + & + & + & + & + \\\hline
  f_{1} & + & + & - & - & + & + & + & + \\\hline
  f_{2} & - & - & + & + & + & + & + & + \\\hline
  f_{3} & + & + & + & + & + & + & + & + \\\hline
  f_{4} & - & - & + & + & + & + & + & + \\\hline
  f_{5} & + & + & + & + & + & + & + & + \\\hline
   & 5 & 4 & 2 & 2 & 0 & 0 & 0 & 0 \\\hline
\end{array}
$"

L"$\left( -2.7 < x < -0.57 \right)$"

### Метод Ньютона

```julia
function newton(f, fp, x0)
    atol = 1e-9
    delta = Inf
    n_iteration = 0
    while (abs(delta) > atol)
        n_iteration += 1
        display("$n_iteration, $x0, $(abs(delta)) > $atol")

        delta = f(x0) / fp(x0)
        x0 = x0 - delta
    end

    x0
end

for t in target_intervals
    mid = t[1] < t[2] ? t[2] + (t[1] - t[2]) / 2 : t[1] + (t[2] - t[1]) / 2
    f_der = derivative(f)
    display("f = $f , f` = $f_der , mid = $mid")

    nn = newton(f, f_der, mid)
    display("x = $nn")
end
```

In [7]:
function newton(f, fp, x0)
    atol = 1e-9
    delta = Inf
    n_iteration = 0
    while (abs(delta) > atol)
        n_iteration += 1
        display("$n_iteration, $x0, $(abs(delta)) > $atol")

        delta = f(x0) / fp(x0)
        x0 = x0 - delta
    end

    x0
end

for t in target_intervals
    mid = t[1] < t[2] ? t[2] + (t[1] - t[2]) / 2 : t[1] + (t[2] - t[1]) / 2
    f_der = derivative(f)
    display("f = $f , f` = $f_der , mid = $mid")

    nn = newton(f, f_der, mid)
    display("x = $nn")
end

"f = 12 - x^3 - 3*x^4 + 4*x^5 , f` = -3*x^2 - 12*x^3 + 20*x^4 , mid = -1.6501428381998404"

"1, -1.6501428381998404, Inf > 1.0e-9"

"2, -1.3682923473596302, 0.28185049084021024 > 1.0e-9"

"3, -1.2093215486996542, 0.15897079865997604 > 1.0e-9"

"4, -1.1591014085399225, 0.05022014015973174 > 1.0e-9"

"5, -1.1546348403831304, 0.004466568156792173 > 1.0e-9"

"6, -1.1546018560112912, 3.29843718392285e-5 > 1.0e-9"

"7, -1.154601854225264, 1.786027057856437e-9 > 1.0e-9"

"x = -1.154601854225264"

### Метод Бісекції

```julia
function bisection(f, a, b)
    atol = 1e-9
    fa = f(a)
    n_iteration = 0
    local c
    while b - a > atol
        c = (a + b) / 2
        fc = f(c)

        n_iteration += 1
        display("$n_iteration, $c, $(b-a) > $atol")

        if fc == 0
            break
        elseif fa * fc > 0
            a = c  # Root is in the right half of [a,b].
            fa = fc
        else
            b = c  # Root is in the left half of [a,b].
        end
    end
    return c
end

for t in target_intervals
    display("f = $f , a = $(t[1]) , b = $(t[2])")
    nn = bisection(f, t[1], t[2])
    display("x = $nn")
end
```

In [8]:
function bisection(f, a, b)
    atol = 1e-9
    fa = f(a)
    n_iteration = 0
    local c
    while b - a > atol
        c = (a + b) / 2
        fc = f(c)

        n_iteration += 1
        display("$n_iteration, $c, $(b-a) > $atol")

        if fc == 0
            break
        elseif fa * fc > 0
            a = c  # Root is in the right half of [a,b].
            fa = fc
        else
            b = c  # Root is in the left half of [a,b].
        end
    end
    return c
end

for t in target_intervals
    display("f = $f , a = $(t[1]) , b = $(t[2])")
    nn = bisection(f, t[1], t[2])
    display("x = $nn")
end

"f = 12 - x^3 - 3*x^4 + 4*x^5 , a = -2.732050807568877 , b = -0.5682348688308035"

"1, -1.6501428381998404, 2.1638159387380735 > 1.0e-9"

"2, -1.109188853515322, 1.0819079693690368 > 1.0e-9"

"3, -1.3796658458575812, 0.5409539846845184 > 1.0e-9"

"4, -1.2444273496864517, 0.2704769923422592 > 1.0e-9"

...

"29, -1.1546018548202612, 8.060842437984661e-9 > 1.0e-9"

"30, -1.1546018528050506, 4.030421107970028e-9 > 1.0e-9"

"31, -1.154601853812656, 2.0152106650073165e-9 > 1.0e-9"

"32, -1.1546018543164585, 1.0076053325036582e-9 > 1.0e-9"

"x = -1.1546018543164585"

### Метод Хорд

```julia
function secant(f, x0, x1)
    atol = 1e-9
    n_iteration = 0
    while abs(x1 - x0) > atol
        y1 = f(x1)
        y0 = f(x0)

        x2 = x0 - y0 * (x1 - x0) / (y1 - y0)

        n_iteration += 1
        display("$n_iteration, $x2, $(abs(x2 - x0)) > $atol")

        x0, x1 = x1, x2
    end
    x1
end

for t in target_intervals
    display("f = $f , a = $(t[1]) , b = $(t[2])")
    nn = secant(f, t[1], t[2])
    display("x = $nn")
end
```

In [9]:
function secant(f, x0, x1)
    atol = 1e-9
    n_iteration = 0
    while abs(x1 - x0) > atol
        y1 = f(x1)
        y0 = f(x0)

        x2 = x0 - y0 * (x1 - x0) / (y1 - y0)

        n_iteration += 1
        display("$n_iteration, $x2, $(abs(x2 - x0)) > $atol")

        x0, x1 = x1, x2
    end
    x1
end

for t in target_intervals
    display("f = $f , a = $(t[1]) , b = $(t[2])")
    nn = secant(f, t[1], t[2])
    display("x = $nn")
end



"f = 12 - x^3 - 3*x^4 + 4*x^5 , a = -2.732050807568877 , b = -0.5682348688308035"

"1, -0.6015671712901529, 2.1304836362787243 > 1.0e-9"

"2, -3.6946402088347097, 3.126405340003906 > 1.0e-9"

"3, -0.6124814906764656, 0.01091431938631271 > 1.0e-9"

"4, -0.6233131596257384, 3.0713270492089713 > 1.0e-9"

...

"19, -1.1545751158986726, 0.01340723876789629 > 1.0e-9"

"20, -1.1546019082591963, 0.001229835485183628 > 1.0e-9"

"21, -1.1546018542228924, 2.673832421984379e-5 > 1.0e-9"

"22, -1.154601854225264, 5.4033932217478764e-8 > 1.0e-9"

"x = -1.154601854225264"